In [ ]:
import json
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.chains import LLMChain
from langchain.llms import LlamaCpp
from langchain.prompts import PromptTemplate

In [ ]:
with open('../data/llm_dataset.json') as f:
    data = json.load(f)

callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

n_gpu_layers = 1
n_batch = 512
llm = LlamaCpp(
    model_path="/Users/scarlettkynoch/Documents/Projects/privfp-gen-experiments/models/quantized_q4_1.gguf",
    n_gpu_layers=n_gpu_layers,
    n_batch=n_batch,
    f16_kv=True,  # MUST set to True, otherwise you will run into problem after a couple of calls
    callback_manager=callback_manager,
    verbose=True
)

In [ ]:
patient_dict = {}
patient_nums = [0, 15, 30, 78, 165, 276, 345, 428, 567, 735, 852, 961]
entities_list = ["name of person",
                  "location of visit",
                  "marital status",
                  #"alcohol consumption",
                  #"allergies", 
                  "male, female or non-binary",
                  "race ethnicity nationality",
                #   "recreational drug use",
                #   "tobacco use",
                  "treatment procedure",
                  "metric and metric value",
                  "medical condition",
                  "medication",
                  "medication dosage",
                  "address",
                  "ID",
                  "NHS Number",
                  "date of birth",
                  "visit date"
                  ]

prompt_template = """A virtual assistant answers questions from a user based on the provided text.
USER: Text: {input_text}
ASSISTANT: I’ve read this text.
USER: What describes {entity_name} in the text?
ASSISTANT: (model's predictions in JSON format)
"""

for patient_num in patient_nums:
    text = data[patient_num].strip()

    patient_entity_dict = {}
    for entity in entities_list:
        prompt = prompt_template.format_map({"input_text": text, "entity_name": entity})
        output = llm(prompt)
        patient_entity_dict[entity] = output

    patient_dict[patient_num] = {
        "text": text,
        "entity_dict": patient_entity_dict 
    }

In [ ]:
prompt_template = """A virtual assistant answers questions from a user based on the provided text.
USER: Text: {input_text}
ASSISTANT: I’ve read this text.
USER: What describes {entity_name} in the text?
ASSISTANT: (model's predictions in JSON format)
"""

patient_nums = [0, 15, 30, 78, 165, 276, 345, 428, 567, 735, 852, 961]
patient_num = patient_nums[5]
text = data[patient_num].strip()
entity = "person full name" # Metric - feed in metric value - get's value. Medication - medication name dosage - you get dosage.

prompt = prompt_template.format_map({"input_text": text, "entity_name": entity})
print(text)
output = llm(prompt)

In [ ]:
print(patient_dict[patient_nums[2]]['text'])
patient_dict[patient_nums[2]]['entity_dict']